In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

#loading data for year 2018
q1_2018 = pd.read_csv('Divvy_Trips_2018_Q1.csv')
q2_2018 = pd.read_csv('Divvy_Trips_2018_Q2.csv')
q3_2018 = pd.read_csv('Divvy_Trips_2018_Q3.csv')
q4_2018 = pd.read_csv('Divvy_Trips_2018_Q4.csv')

#Q1 data has different column names , making them consistent accross all the data frames by renaming
q1_2018=q1_2018.rename(columns={'01 - Rental Details Rental ID':'trip_id',
'01 - Rental Details Local Start Time':'start_time',
'01 - Rental Details Local End Time':'end_time',
'01 - Rental Details Bike ID':'bikeid',
'01 - Rental Details Duration In Seconds Uncapped':'tripduration',
'03 - Rental Start Station ID':'from_station_id',
'02 - Rental End Station ID':'to_station_id',
'03 - Rental Start Station Name':'from_station_name',
'02 - Rental End Station Name':'to_station_name',
'User Type':'usertype',
'Member Gender':'gender',
'05 - Member Details Member Birthday Year':'birthyear'})

#COmbinig 2018 data to single df
data_2018 = pd.concat([q1_2018,q2_2018,q3_2018,q4_2018],axis=0)

#Converting to datetime
data_2018.start_time = pd.to_datetime(data_2018.start_time)
data_2018.end_time  = pd.to_datetime(data_2018.end_time)



In [ ]:
#cleaning trip duration columns for values like 2,904.0
data_2018['tripduration'] = (data_2018.end_time - data_2018.start_time).dt.seconds
#Extracting  month ,day ,hour ,date from start_time
data_2018['month'] = data_2018.start_time.dt.month
data_2018['date']   = data_2018.start_time.dt.date
data_2018['hour'] = data_2018.start_time.dt.hour
data_2018['day'] = data_2018.start_time.dt.day
data_2018['DayofWeek'] = data_2018.start_time.dt.dayofweek

#Finding is a day is a weekend
data_2018['weekend']=np.where(data_2018.start_time.dt.weekday > 4 , 1, 0)

#Calculating age in 2018 and dropping birthyear
data_2018['age'] = np.where(data_2018.birthyear > 0 ,2018 - data_2018['birthyear'],0)
data_2018.drop(columns='birthyear',inplace=True)
#shape (3603082, 17)

#dropping observations with age >80 as riders cannot be more than age of 80
data_2018=data_2018[data_2018['age'] <80]
# shape (3601532, 17)

#dropping observatoins with Subscribers having age 0
data_2018['age_']=(np.logical_and(data_2018['usertype']=='Subscriber',data_2018['age'] == 0))
data_2018 = data_2018[data_2018.age_ ==False]
data_2018.drop(columns='age_',inplace=True)
#shape (3598317, 17)

#Final Trip details for 2018
data_2018.to_csv('data_2018.csv')

In [ ]:
#importing weather data for 2018 chicago
weather = pd.read_csv('weather1.csv')

#Snow details are not available on hourly level only day level saving to new df
snow_day = weather[weather.REPORT_TYPE == 'SOD  ']


#cleaning weather data 
weather = weather[weather.REPORT_TYPE != 'SOD  '] #snow day
weather = weather[weather.REPORT_TYPE != 'SOM  '] #snow month 
weather = weather[weather.REPORT_TYPE != 'SY-MT'] 
weather = weather[weather.REPORT_TYPE != 'FM-12'] 
weather = weather[weather.REPORT_TYPE != 'FM-16'] 


#Keeping only relevant columns in weather
weather = weather[['DATE','HourlyDryBulbTemperature','HourlyPrecipitation']]
weather.rename(columns={'HourlyDryBulbTemperature':'HourlyTemperature'},inplace=True)

#Extracting  month ,day ,hour from DATE column
weather['month'] = pd.to_datetime(weather.DATE,format='%Y-%m-%d').dt.month
weather['day'] = pd.to_datetime(weather.DATE,format='%Y-%m-%d').dt.day
weather['hour']=pd.to_datetime(weather.DATE,format='%Y-%m-%d').dt.hour
weather.drop(columns='DATE',inplace=True)


#converting HourlyPrecipitation with value 'T'traces to 0.01 and nan with ffill


weather['HourlyPrecipitation'] = np.where(weather['HourlyPrecipitation'] == 'T',0.01,weather['HourlyPrecipitation'])
weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].astype(float)

weather['HourlyPrecipitation']=weather['HourlyPrecipitation'].fillna(method='ffill')

weather['HourlyTemperature'] = weather['HourlyTemperature'].fillna(method='ffill')

#Cleaned weather data 2018
weather.to_csv('weather_final.csv')

#selecting relevent columns from snow_day data frame
snow_day = snow_day[['DATE','DailySnowDepth']]

#Extracting date from DATE 
snow_day['date'] = pd.to_datetime(snow_day.DATE,format='%Y-%m-%d').dt.date
snow_day['month']= pd.to_datetime(snow_day.DATE,format='%Y-%m-%d').dt.month
snow_day['day'] = pd.to_datetime(snow_day.DATE,format='%Y-%m-%d').dt.day
snow_day.drop(columns='DATE',inplace=True)
snow_day.to_csv('snow.csv')



In [ ]:
#Merging trip details with snow_day , this gives amount of DailySnowDepth for each day
data_merged = data_2018.merge(snow_day ,on=['date','month','day'])
#shape (3598317, 18)

In [ ]:
#Merging trip details with weather adding HourlyTemperature and HourlyPrecipitation
data_merged = data_merged.merge(weather,on=['month','day','hour'],how='inner')
#Shape (3588777, 19)


#dropping previous index column
data_merged.drop(data_merged.iloc[:,0:1],axis=1,inplace=True)

In [ ]:
data_merged = data_2018.merge(snow_day ,on='date')
#Shape(3598317, 17)

In [ ]:
data_merged.tail()

In [ ]:
#Merging trip details with weather adding HourlyTemperature and HourlyPrecipitation
data_merged = data_merged.merge(weather,on=['month','day','hour'],how='inner')
#Shape (3588777, 20)

In [ ]:
#exporting final preprocessed data  
data_merged.to_csv('data_merged.csv')